In [10]:
import awswrangler as wr

import mlflow

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


# Búsqueda de mejor modelo e hiperparámetros

Dado nuestro dataset, el cual ya pasó por el proceso de ETL y se encuentra en nuestro S3 bucket, vamos a realizar una búsqueda de cual seria el mejor modelo y que hiperparametros usar.

La búsqueda de hiperparametros la haremos usando Optuna y el tracking será realizado mediante MLFlow.

OBS: Para la confección de esta notebook, nos basamos en el tutorial de [MLFlow](https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/index.html).

In [11]:
mlflow_server = "http://localhost:5000"
#mlflow_server = "http://192.168.0.21:5000"

mlflow.set_tracking_uri(mlflow_server)

In [12]:
# Cargamos los datos para realizar nuestro estudio.
# OBS, no vamos a cargar los datos de testing, nada de Data leakage por aquí
X_train = wr.s3.read_csv("s3://data/final/train/star_X_train.csv")
y_train = wr.s3.read_csv("s3://data/final/train/star_y_train.csv")

X_test = wr.s3.read_csv("s3://data/final/test/star_X_test.csv")
y_test = wr.s3.read_csv("s3://data/final/test/star_y_test.csv")


In [13]:
print(X_train)

         obj_ID     alpha     delta         u         g         r         i  \
0     -0.151403  1.506682 -1.229315 -0.027928 -0.042080 -0.968742 -0.945357   
1      1.721125 -1.471247 -1.433494 -0.000849  0.012195  0.704779  0.932464   
2     -1.097912  0.808174  0.425162  0.088323  0.073398  1.037008  0.767655   
3      1.844968  1.798912 -0.070862  0.002050  0.021201  0.218118  0.134464   
4      0.293375 -0.112060  0.167313 -0.060604 -0.047515 -0.691826 -0.472763   
...         ...       ...       ...       ...       ...       ...       ...   
69995  0.773724 -0.472021 -0.652333  0.036142  0.001115 -0.474954 -0.524982   
69996 -0.469664 -0.254866  0.802815 -0.072694 -0.037749 -0.290759 -0.096327   
69997 -0.641045  0.352595  1.751331  0.038174  0.049127  0.645290  0.447952   
69998 -0.151276  1.497554 -1.180462  0.051178  0.050648  0.532544  0.435954   
69999  1.643676  1.862628 -0.354669  0.012606  0.059742  0.715077  0.485265   

              z    run_ID   cam_col  field_ID  spec

## Investigamos la correlación de features con la variable objetivo

Antes de profundizar en el proceso de construcción de modelo, es esencial comprender las relaciones entre nuestras features  y la variable objetivo. Por lo que vamos a realizar un gráfico que indica el coeficiente de correlación de cada feature en relación con la variable objetivo. Esto nos sirve para:

- Evitar data leakage: Debemos asegurarnos de que ninguna característica se correlacione perfectamente con el objetivo (un coeficiente de correlación de aproximadamente 1.0). Si existe tal correlación, es una señal de que nuestro conjunto de datos podría estar "filtrando" información sobre el objetivo. 

- Garantizar relaciones significativas: Idealmente, nuestras características deberían tener algún grado de correlación con el objetivo. Inclusive si estamos trabajando con un problema de clasificación, aunque los resultados no son tan importantes como en un caso de regresión.

- Auditoría y trazabilidad: Loggear esta visualización de correlación con nuestra ejecución principal de MLflow garantiza la trazabilidad. Proporciona una instantánea de las características de los datos en el momento del entrenamiento del modelo, lo cual es invaluable para propósitos de auditoría y replicabilidad.

In [14]:
from plots import plot_correlation_with_target, plot_information_gain_with_target

In [15]:
# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
correlation_plot = plot_correlation_with_target(X_train, y_train)
information_gain_plot = plot_information_gain_with_target(X_train, y_train)

KeyboardInterrupt: 

## Arrancamos a experimentar

In [ ]:
import datetime
import optuna

from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment

from optuna_aux import champion_callback, objective

from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

/home/jorge/miniconda3/envs/mlops-5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [ ]:
# Creemos el experimento
experiment_id = get_or_create_experiment("Star Classification")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


Ya con todo seteado, vamos a ejecutar la optimización usando Optuna, el cual realiza una búsqueda Bayesiana, la cual es más eficiente que una búsqueda de grilla tradicional. La desventaja es que es más difícil de paralelizar.

In [ ]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Inicializamos el estudio de Optuna
    study = optuna.create_study(direction="maximize")

    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=250, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Star Classification",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos
    if study.best_params["classifier"] == "SVC_linear":
        model = SVC(C=study.best_params["svc_c"], kernel='linear', gamma='scale')
    elif study.best_params["classifier"] == "SVC_poly":
        model = SVC(C=study.best_params["svc_c"], kernel='poly', 
                    gamma='scale', degree=study.best_params["svc_poly_degree"])
    elif study.best_params["classifier"] == "SVC_rbf":
        model = SVC(C=study.best_params["svc_c"], kernel='rbf', gamma='scale')
    elif study.best_params["classifier"] == "DecisionTreeClassifier":
        model = DecisionTreeClassifier(max_depth=study.best_params["tree_max_depth"])
    else:
        model = RandomForestClassifier(max_depth=study.best_params["rf_max_depth"], 
                                       n_estimators=study.best_params["rf_n_estimators"])

    model = model.fit(X_train, y_train.to_numpy().ravel())

    # Y testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    #f1_score = f1_score(y_test.to_numpy().ravel(), y_pred)
    f1_score = f1_score(y_test.to_numpy().ravel(), y_pred, average='macro')
    mlflow.log_metric("test_f1", f1_score)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="star_class_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)


🏃 View run Trial: 0 at: http://localhost:5000/#/experiments/1/runs/ea0859712c644599b29ffb5487a10961
🧪 View experiment at: http://localhost:5000/#/experiments/1
Initial trial 0 achieved value: 0.24877470274506153
🏃 View run Trial: 1 at: http://localhost:5000/#/experiments/1/runs/ecb9655464f14e05aa69e76091eb972d
🧪 View experiment at: http://localhost:5000/#/experiments/1
Trial 1 achieved value: 0.2493072056330873 with  0.2136% improvement
🏃 View run Trial: 2 at: http://localhost:5000/#/experiments/1/runs/3fb22c871fc74ee08ff6f9d60a607697
🧪 View experiment at: http://localhost:5000/#/experiments/1
🏃 View run Trial: 3 at: http://localhost:5000/#/experiments/1/runs/f28773f497d84e06b6d6237edddaf8c3
🧪 View experiment at: http://localhost:5000/#/experiments/1
Trial 3 achieved value: 0.27600799896687034 with  9.6739% improvement
🏃 View run Trial: 4 at: http://localhost:5000/#/experiments/1/runs/0fd5b0e088404e16bfafb8e4204258d2
🧪 View experiment at: http://localhost:5000/#/experiments/1
🏃 View ru

KeyboardInterrupt: 

## Testeando el modelo

Una vez que el modelo fue entrenado, podemos levantarlo y testearlo de una forma agnóstica a donde está guardado.

In [ ]:
loaded = mlflow.sklearn.load_model(model_uri)

In [ ]:
import numpy as np
test_data = [0.741830344520968, -1.3390058817968316, 0.9270742100032214, -0.022594692703017016, -0.020158498007864664,
             -0.32633765093444494, -0.16166633313766, 0.0006512010351584627, 0.7417317177335456, 0.9400630553605082, 
             0.2899351799963413, -0.9217758867440433, -0.788211578207859, -0.9217809160581861, -1.0088989161501674, 0.0013417780822043853]

loaded.predict(np.array(test_data).reshape([1, -1]))

## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
name = "star_class_model_prod"
desc = "This is a star classifier model "

# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1-score"] = f1_score

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)